# Types and Dispatch in Julia

Although types aren't always explicitly visible, **Julia is built around types**.

High performance codes in Julia make good use of the type system.

## Abstract vs concrete types

**Concrete types**
* types of values ("objects")
* specify data structure

**Abstract types**
* cannot be instantiated
* define sets of concrete types (their descendants) by their shared "behavior" (duck typing)

In [2]:
3 + 2.0

5.0

In [3]:
typeof(3)

Int64

In [5]:
typeof(2.0)

Float64

In [6]:
isconcretetype(Float64)

true

In [7]:
3 isa Int64

true

In [8]:
3 isa Float64

false

In [9]:
3 isa Number

true

In [10]:
isabstracttype(Number)

true

In [11]:
isabstracttype(Real)

true

### [Duck typing](https://en.wikipedia.org/wiki/Duck_typing)

**"If it walks like a duck and it quacks like a duck, then it must be a duck"**

The abstract type `Number` indicates that one can do number-like things, e.g. `+`,`-`,`*`, and `/`, with corresponding values. In this category we have (concrete) things like `Float64` and `Int32` numbers.

An `AbstractArray` is a type that, e.g., allows indexing `A[i]`. Examples include regular arrays (`Array`), as well as ranges (`UnitRange`).

## Inspecting the type tree

In [17]:
supertype(Float64)

AbstractFloat

In [18]:
supertype(AbstractFloat)

Real

In [19]:
subtypes(AbstractFloat)

4-element Vector{Any}:
 BigFloat
 Float16
 Float32
 Float64

In [20]:
supertype(Real)

Number

In [21]:
supertype(Number)

Any

Everything is a subtype of `Any`

In [22]:
Number <: Any

true

In [23]:
Float64 <: Any

true

In [24]:
Int32 <: Any

true

In [25]:
Int32 <: String

false

Let's extract a branch of the type tree and visualize it

In [29]:
using AbstractTrees
AbstractTrees.children(x) = subtypes(x)

In [30]:
print_tree(Number)

Number
├─ Complex
└─ Real
   ├─ AbstractFloat
   │  ├─ BigFloat
   │  ├─ Float16
   │  ├─ Float32
   │  └─ Float64
   ├─ AbstractIrrational
   │  └─ Irrational
   ├─ Integer
   │  ├─ Bool
   │  ├─ Signed
   │  │  ├─ BigInt
   │  │  ├─ Int128
   │  │  ├─ Int16
   │  │  ├─ Int32
   │  │  ├─ Int64
   │  │  └─ Int8
   │  └─ Unsigned
   │     ├─ UInt128
   │     ├─ UInt16
   │     ├─ UInt32
   │     ├─ UInt64
   │     └─ UInt8
   └─ Rational


Note that **concrete types are the leaves of the type tree** whereas **abstract types are nodes** in the type graph.

## Functions, methods, and dispatch

Let's define a *function* that calculates the absolute value of a number (like the built-in `abs` already does).

**How would we practically calculate the absolute values of the numbers $-4.32$ and $1.0 + 1.0i$?**

* Real number
  * "Drop the sign."
    * => `myabs(-4.32) = 4.32`
* Complex number:
  * "Square root of z times the complex conjugate of z."
    * => `myabs(1.0 + 1.0im) = sqrt(2) ≈ 1.414`

We see that the *methods* that we use depend on the type of the number.

While the single **function** represents the *what* ("calculate the absolute value"), there might be different **methods** describing the *how*.

We can use the `::` operator to annotate function arguments with types and define different methods.

In [31]:
myabs(x::Float64) = sign(x) * x

myabs (generic function with 1 method)

In [32]:
myabs(-4.32)

4.32

In [33]:
myabs(1.0 + 1.0im)

LoadError: MethodError: no method matching myabs(::ComplexF64)
[0mClosest candidates are:
[0m  myabs([91m::Float64[39m) at In[31]:1

In [34]:
myabsthatdoesntexist(1.0 + 1.0im)

LoadError: UndefVarError: myabsthatdoesntexist not defined

In [35]:
myabs(z::ComplexF64) = sqrt(real(z * conj(z)))

myabs (generic function with 2 methods)

In [36]:
myabs(1.0 + 1.0im)

1.4142135623730951

In [37]:
methods(myabs)

# 2 methods for generic function "myabs":
[1] myabs(x::Float64) in Main at In[31]:1
[2] myabs(z::ComplexF64) in Main at In[35]:1

One can check which particular method is being used through the `@which` macro.

In [38]:
@which myabs(-4.32)

myabs(x::Float64) in Main at In[31]:1

In [39]:
@which myabs(1.0 + 1.0im)

myabs(z::ComplexF64) in Main at In[35]:1

Note that we should better loosen our type restrictions:

In [40]:
myabs(-3)

LoadError: MethodError: no method matching myabs(::Int64)
[0mClosest candidates are:
[0m  myabs([91m::Float64[39m) at In[31]:1
[0m  myabs([91m::ComplexF64[39m) at In[35]:1

In [41]:
myabs(1 + 1im)

LoadError: MethodError: no method matching myabs(::Complex{Int64})
[0mClosest candidates are:
[0m  myabs([91m::Float64[39m) at In[31]:1
[0m  myabs([91m::ComplexF64[39m) at In[35]:1

In [42]:
myabs(x::Real) = sign(x) * x
myabs(z::Complex) = sqrt(real(z * conj(z)))

myabs (generic function with 4 methods)

In [43]:
myabs(-3)

3

As we will understand later, type annotations in function signatures virtually never affect performance!

**One should therefore generally make them as generic as possible.**

### Multiple dispatch

Which method gets executed when you call a generic function `f` for a given set of input arguments?

**Answer:** Julia always chooses the **most specific method** by considering **all input argument types**.

(Since methods belong to generic functions rather than objects no function argument is special.)

In [45]:
f(a, b::Any)              = "fallback"
f(a::Number, b::Number)   = "a and b are both numbers"
f(a::Number, b)           = "a is a number"
f(a, b::Number)           = "b is a number"
f(a::Integer, b::Integer) = "a and b are both integers"

f (generic function with 5 methods)

In [46]:
methods(f)

# 5 methods for generic function "f":
[1] f(a::Integer, b::Integer) in Main at In[45]:5
[2] f(a::Number, b::Number) in Main at In[45]:2
[3] f(a::Number, b) in Main at In[45]:3
[4] f(a, b::Number) in Main at In[45]:4
[5] f(a, b) in Main at In[45]:1

In [47]:
f(1.5, 2)

"a and b are both numbers"

In [48]:
f(1, "Stuttgart!")

"a is a number"

In [49]:
f(1, 2)

"a and b are both integers"

In [50]:
f("Hello", "World!")

"fallback"

In [51]:
@which f(1, 2)

f(a::Integer, b::Integer) in Main at In[45]:5

In [52]:
@which f(1, "Stuttgart!")

f(a::Number, b) in Main at In[45]:3

It happens rarely, but it can happen that there is no unique most specific method:

In [72]:
f(x::Int, y::Any) = println("int")
f(x::Any, y::String) = println("string")
f(3, "test")

LoadError: MethodError: f(::Int64, ::String) is ambiguous. Candidates:
  f(x::Int64, y) in Main at In[72]:1
  f(a::Number, b) in Main at In[45]:3
  f(x, y::String) in Main at In[72]:2
Possible fix, define
  f(::Int64, ::String)

### Built-in Julia function

(Most of) **Julia's built-in functions are not special by any means.**

In [53]:
methods(+)

# 206 methods for generic function "+":
[1] +(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:87
[2] +(x::T, y::T) where T<:Union{Float16, Float32, Float64} in Base at float.jl:383
[3] +(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigInt) in Base.GMP at gmp.jl:531
[4] +(c::Union{Int16, Int32, Int64, Int8}, x::BigInt) in Base.GMP at gmp.jl:537
[5] +(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigFloat) in Base.MPFR at mpfr.jl:398
[6] +(c::Union{Int16, Int32, Int64, Int8}, x::BigFloat) in Base.MPFR at mpfr.jl:406
[7] +(c::Union{Float16, Float32, Float64}, x::BigFloat) in Base.MPFR at mpfr.jl:414
[8] +(x::Union{Dates.CompoundPeriod, Dates.Period}) in Dates at /Users/carstenbauer/.julia/juliaup/julia-1.8.0+0.aarch64/share/julia/stdlib/v1.8/Dates/src/periods.jl:378
[9] +(A::LinearAlgebra.Symmetric, B::LinearAlgebra.Symmetric) in LinearAlgebra at /Users/carstenbauer/.julia/juliaup/julia-1.8.0+0.aarch64/share/julia/stdlib/v1.8/LinearAlgebra/src/symmetric.jl:469
[10] +(A::LinearAlgebra.Symmetric, B::LinearAlgebra.SymTridiagonal) in LinearAlgebra at /Users/carstenbauer/.julia/juliaup/julia-1.8.0+0.aarch64/share/julia/stdlib/v1.8/LinearAlgebra/src/symmetric.jl:486
[11] +(S::LinearAlgebra.Symmetric, D::LinearAlgebra.Diagonal) in LinearAlgebra at /Users/carstenbauer/.julia/juliaup/julia-1.8.0+0.aarch64/share/julia/stdlib/v1.8/LinearAlgebra/src/diagonal.jl:191
[12] +(A::LinearAlgebra.Symmetric{<:Any, <:SparseArrays.AbstractSparseMatrix}, B::SparseArrays.AbstractSparseMatrix) in SparseArrays at /Users/carstenbauer/.julia/juliaup/julia-1.8.0+0.aarch64/share/julia/stdlib/v1.8/SparseArrays/src/linalg.jl:15
[13] +(A::LinearAlgebra.Symmetric, B::SparseArrays.AbstractSparseMatrix) in SparseArrays at /Users/carstenbauer/.julia/juliaup/julia-1.8.0+0.aarch64/share/julia/stdlib/v1.8/SparseArrays/src/linalg.jl:18
[14] +(A::LinearAlgebra.Symmetric{<:Real, <:SparseArrays.AbstractSparseMatrix}, B::LinearAlgebra.Hermitian{<:Any, <:SparseArrays.AbstractSparseMatrix}) in SparseArrays at /Users/carstenbauer/.julia/juliaup/julia-1.8.0+0.aarch64/share/julia/stdlib/v1.8/SparseArrays/src/linalg.jl:25
[15] +(A::LinearAlgebra.Symmetric{var"#s883", S} where {var"#s883"<:Real, S<:(AbstractMatrix{<:var"#s883"})}, B::LinearAlgebra.Hermitian) in LinearAlgebra at /Users/carstenbauer/.julia/juliaup/julia-1.8.0+0.aarch64/share/julia/stdlib/v1.8/LinearAlgebra/src/symmetric.jl:484
[16] +(A::LinearAlgebra.Symmetric{<:Any, <:SparseArrays.AbstractSparseMatrix}, B::LinearAlgebra.Hermitian{<:Any, <:SparseArrays.AbstractSparseMatrix}) in SparseArrays at /Users/carstenbauer/.julia/juliaup/julia-1.8.0+0.aarch64/share/julia/stdlib/v1.8/SparseArrays/src/linalg.jl:23
[17] +(F::LinearAlgebra.Hessenberg, J::LinearAlgebra.UniformScaling) in LinearAlgebra at /Users/carstenbauer/.julia/juliaup/julia-1.8.0+0.aarch64/share/julia/stdlib/v1.8/LinearAlgebra/src/hessenberg.jl:641
[18] +(A::LinearAlgebra.Hermitian, B::LinearAlgebra.Hermitian) in LinearAlgebra at /Users/carstenbauer/.julia/juliaup/julia-1.8.0+0.aarch64/share/julia/stdlib/v1.8/LinearAlgebra/src/symmetric.jl:469
[19] +(A::LinearAlgebra.Hermitian, B::LinearAlgebra.SymTridiagonal{var"#s881", V} where {var"#s881"<:Real, V<:AbstractVector{var"#s881"}}) in LinearAlgebra at /Users/carstenbauer/.julia/juliaup/julia-1.8.0+0.aarch64/share/julia/stdlib/v1.8/LinearAlgebra/src/symmetric.jl:488
[20] +(H::LinearAlgebra.Hermitian, D::LinearAlgebra.Diagonal{var"#s884", V} where {var"#s884"<:Real, V<:AbstractVector{var"#s884"}}) in LinearAlgebra at /Users/carstenbauer/.julia/juliaup/julia-1.8.0+0.aarch64/share/julia/stdlib/v1.8/LinearAlgebra/src/diagonal.jl:197
[21] +(A::LinearAlgebra.Hermitian, J::LinearAlgebra.UniformScaling{<:Complex}) in LinearAlgebra at /Users/carstenbauer/.julia/juliaup/julia-1.8.0+0.aarch64/share/julia/stdlib/v1.8/LinearAlgebra/src/uniformscaling.jl:195
[22] +(A::LinearAlgebra.Hermitian{<:Any, <:SparseArrays.AbstractSparseMatrix}, B::SparseArrays.Abstract

In [54]:
@which true + false

+(x::Bool, y::Bool) in Base at bool.jl:162

In [55]:
@which "Hello"*"World!"

*(s1::Union{AbstractChar, AbstractString}, ss::Union{AbstractChar, AbstractString}...) in Base at strings/basic.jl:260

We can easily modify or add methods to them as well.

In [73]:
import Base: + # we have to import functions to override/add methods
+(x::String, y::String) = x * "_" * y

# alternative
Base.:+(x::String, y::String) = x * "_" * y

In [74]:
"Hello" + "Stuttgart!"

"Hello_Stuttgart!"

(**Side note**: as we neither own the `+` function nor the `String` type the above is **type piracy** and should generally be avoided! 😉)

# Type Parameters

Types can have *type parameters*. They are crucial for achieving high performance while being generic at the same time (more on this later).

The most prominent example is Julia's regular array type.

In [83]:
M = rand(2,2)

2×2 Matrix{Float64}:
 0.646114  0.990087
 0.22244   0.624745

In [84]:
typeof(M)

Matrix{Float64} (alias for Array{Float64, 2})

Here, `Array` is a parametric datatype. The type parameters are
* `Float64` (element type)
* `2` (dimensionality)

Hence `Array{Float64, 2}` means that we have a matrix than can hold 64-bit floating point numbers.

This generalizes as expected. Here, a vector of `String`s:

In [77]:
M = fill("Stuttgart", 2)

2-element Vector{String}:
 "Stuttgart"
 "Stuttgart"

In [78]:
eltype(M)

String

We can also nest parametric types. This is a vector of matrices of `Float64`s.

In [79]:
v = [rand(2,2) for i in 1:3]

3-element Vector{Matrix{Float64}}:
 [0.161652025216949 0.016571056847715426; 0.41737802368459276 0.5302870433648723]
 [0.6928672296607381 0.20408572435277972; 0.976260108428621 0.41895139337374276]
 [0.9335004121752404 0.8038561628199073; 0.881214768681139 0.8706975597631867]

In [80]:
eltype(v)

Matrix{Float64} (alias for Array{Float64, 2})

Another example of a parametric type is the `Tuple`.

In [81]:
(1,2.0,"3")

(1, 2.0, "3")

In [82]:
typeof((1,2.0,"3"))

Tuple{Int64, Float64, String}

### Type parameters in function signatures

Naive approach:

In [85]:
myfunc(v::Vector{Real}) = "I'm a real vector!"

myfunc (generic function with 1 method)

In [86]:
myfunc([1.0,2.0,3.0])

LoadError: MethodError: no method matching myfunc(::Vector{Float64})
[0mClosest candidates are:
[0m  myfunc([91m::Vector{Real}[39m) at In[85]:1

Huh? What's going on?

Note that although we have

In [ ]:
Float64 <: Real

parametric types have the following (perhaps somewhat counterintuitive) property

In [87]:
Vector{Float64} <: Vector{Real}

false

In [88]:
[1.0,2.0,3.0] isa Vector{Real}

false

How can we understand the behavior above?

The crucial point is that `Vector{Real}` is a **concrete** container type despite the fact that `Real` is an abstract type. Specifically, it describes a **heterogeneous** vector of values that individually can be of any type `T <: Real`.

In [89]:
isconcretetype(Vector{Real})

true

In [90]:
Real[1, 2.2, 13f0]

3-element Vector{Real}:
  1
  2.2
 13.0f0

As we have learned above, concrete types are the leafes of the type tree and **cannot** have any subtypes. Hence it is only consistent to have...

In [91]:
Vector{Float64} <: Vector{Real}

false

What we often actually *mean* when writing `myfunc(v::Vector{Real}) = ...` is

In [ ]:
myfunc(v::Vector{T}) where T<:Real = "I'm a real vector!"

In [ ]:
myfunc([1.0,2.0,3.0])

It works! But what does it mean exactly? First of all, we see that

In [ ]:
Vector{Float64} <: Vector{T} where T<:Real

Here, `Vector{T} where T <: Real` describes the **set** of concrete `Vector` types whose elements are of any specific single type `T` that is a subtype of `Real`.

Think of it as representing

`{{ Vector{Float64}, Vector{Int64}, Vector{Int32}, Vector{AbstractFloat}, ... }}`

where we use double curly braces to indicate the set.

In [92]:
Vector{Int64} <: Vector{T} where T<:Real

true

In [93]:
Vector{AbstractFloat} <: Vector{T} where T<:Real

true

In [94]:
[1.0,2.0,3.0] isa Vector{T} where T<:Real

true

We can also use the `where` notation to write out our naive `Vector{Real}` from above in a more explicit way:

In [95]:
Vector{Real} === Vector{T where T<:Real}

true

Note that the crucial difference is the position of the `where T<:Real` piece, i.e. whether it is inside or outside of the curly braces.

In [96]:
Vector{T where T<:Real} <: Vector{T} where T<:Real

true

In [97]:
(Vector{T} where T<:Real) <: Vector{T where T<:Real}

false

(More mathematically put: Whether `where T` is inside our outside of the curly braces indicates whether there is or is not a "degree of freedom" that spans the "one-dimensional" set above.)

# Core messages of this notebook

* **Concrete types** describe data structures, i.e. concrete implementations.
* **Abstract types** define the kind of a thing (What is it? What can I do with it?), i.e. an informal interface. This is also known as **duck-typing**.
* A **function** (the what) can have multiple **methods** (the how).
* Types in function signatures serve as filters. **Avoid writing overly-specific types**.
* **Multiple dispatch**: Julia selects the method to run based on the types of all input arguments and chooses the most specialized one.
* Types can have parameters, i.e. `Vector{Float64}`. We can use the notation `T where T<:SomeSuperType` to address *sets* of types.